<h1><center>Farrukh's Week 3 - Assignment Part 1</center></h1>

### Loading Dependinces

In [40]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## Visulaziations
from geopy.geocoders import Nominatim
import folium
import time

## KMeans
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
print("Loaded all required Librarires!!!")

Loaded all required Librarires!!!


###  Scraping Canada's Data
#### Method: BeautifulSoup

1. Reading all page content
2. Searching for a table
3. Searching for a header and collecting all column headers to transform them into our dataframe headers
4. Building a dictionary with each column content collected into a list ( on the go taking only text content avoiding any markups tags)

In [25]:
url_to_scrape = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url_to_scrape)
text = []
if page.status_code==200:
    soup = BeautifulSoup(page.content, 'html.parser')
    
    data = {}
    headers = []
    for row in soup.findAll('table')[0].tbody.findAll('tr'):
        if len(row.findAll('th'))>0:
            for h in row.findAll('th'):
                hdr = h.get_text(strip=True)           
                headers.append(hdr)
                data[hdr] = []
        if len(row.findAll('td'))>0:
            for i in range (len(headers)):
                data[headers[i]].append(row.findAll('td')[i].get_text(strip=True))

Convert scrapped data structure into pandas DataFrame and rename columns as required 

In [26]:
toronto_nbd = pd.DataFrame(data)
toronto_nbd = toronto_nbd.rename(columns = {'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'})

Now Check how many Neighborhoods are set to value "Not Assigned"

In [54]:
not_assigned = toronto_nbd.loc[toronto_nbd['Neighborhood']=='Not assigned'].values.shape[0]
print("Not Assigned (Neighborhoods) :", not_assigned)

Not Assigned (Neighborhoods) : 77


####  Replacing 'Not assigned' neighbouthood values with values of 'Borough' column

In [55]:
toronto_nbd['Neighborhood'] = toronto_nbd.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)

In [56]:
toronto_nbd.loc[toronto_nbd['Neighborhood']=='Not assigned'].values.shape[0]

77

We have done the replacement now, its only one replacment

##### Joining Neibourghoods having same PostalCode value as required - in a coma separated list of strings

The result goes into result_toronto DataFrame, where we will be still mising the Borough column, but that will come later

In [57]:
grouped_nbd = toronto_nbd.groupby('PostalCode')['Neighborhood'].agg(lambda x: ','.join(x))
toronto_df = pd.DataFrame()
toronto_df['PostalCode'] = grouped_nbd.index
toronto_df['Neighborhood'] = grouped_nbd.values
toronto_df = toronto_df.sort_values(by = 'PostalCode')
toronto_df.head()

,PostalCode,Neighborhood
0,M1A,Not assigned
1,M1B,"Rouge,Malvern"
2,M1C,"Highland Creek,Rouge Hill,Port Union"
3,M1E,"Guildwood,Morningside,West Hill"
4,M1G,Woburn


# Now we can use values from the source table for Borough and add then to the result table via left merge on PostalCode

In [58]:
toronto_nbd.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [59]:
toronto_df = toronto_df.merge(toronto_nbd[['PostalCode', 'Borough']].drop_duplicates(), on = 'PostalCode', how = 'left', copy='False' )

In [60]:
toronto_df.head()

,PostalCode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge,Malvern",Scarborough
2,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
3,M1E,"Guildwood,Morningside,West Hill",Scarborough
4,M1G,Woburn,Scarborough


In [61]:
toronto_df.loc[toronto_df['PostalCode']=='M5A']

,PostalCode,Neighborhood,Borough
80,M5A,"Harbourfront,Regent Park",Downtown Toronto


In [62]:
toronto_df.shape

(180, 3)

# Analyze resulting data frame and eliminating  all 'Not assigned' and explicit NaNs

In [63]:
toronto_df.describe(include='all')

,PostalCode,Neighborhood,Borough
count,180,180,180
unique,180,104,12
top,M9N,Not assigned,Not assigned
freq,1,77,77


In [64]:
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].replace('Not assigned', np.NaN)
toronto_df['Borough'] = toronto_df['Borough'].replace('Not assigned', np.NaN)
toronto_df = toronto_df.dropna()
toronto_df.shape

(103, 3)

In [65]:
print("Check for NA")
toronto_df.isna().sum()

Check for NA


PostalCode      0
Neighborhood    0
Borough         0
dtype: int64

## Presist the Dataframe to file for next Part Usage

In [67]:
toronto_df.to_csv('toronto_data.scv')

## Shape

In [69]:
toronto_df.shape

(103, 3)